# Temperature Exploration

August 11, 2022  
Gully & Ryan H.

Using Gollum to explore different stars and their temperatures.

In [ ]:
from gollum.phoenix import PHOENIXGrid, PHOENIXSpectrum
from specutils import Spectrum1D
import pandas as pd
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from urllib.error import URLError
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
path='..\\..\\..\\data\\phoenix.astro.physik.uni-goettingen.de\\HiResFITS\\PHOENIX-ACES-AGSS-COND-2011\\'

In [ ]:
df='../../data/Rapid_Rotator_Sample.csv'

In [ ]:
grid = PHOENIXGrid(teff_range=(5000, 7000), logg_range=(2, 5), metallicity_range=(-0.5, 0), wl_lo= 4000, wl_hi= 12000,
                   path=path)

In [ ]:
grid.truncate

## Fetch the filter curve

In [ ]:
tess_filter_curve_website = 'https://heasarc.gsfc.nasa.gov/docs/tess/data/tess-response-function-v1.0.csv'
local_file = '../../data/tess-response-function-v1.0.csv'

In [ ]:
# try to directly read from the website first, if not then download the file locally
try:
    tess_response = pd.read_csv(tess_filter_curve_website, skiprows=8, names=['wl_nm', 'transmission'])
except URLError:
    tess_response = pd.read_csv(local_file, skiprows=8, names=['wl_nm', 'transmission'])

In [ ]:
TESS_filter_curve = Spectrum1D(spectral_axis=tess_response.wl_nm.values*u.nm, 
           flux=tess_response.transmission.values*u.dimensionless_unscaled)

Now get the Kepler filter curve

In [ ]:
kep_response = pd.read_csv('../../data/kepler_response_hires1.txt',
                           names=['wl_nm','transmission'],
                           skiprows=9, delim_whitespace=True)
kep_response['wl_ang'] = kep_response.wl_nm * 10.0
kep_response = kep_response[(kep_response.wl_ang > 4200) & (kep_response.wl_ang < 9200)]
kep_response['trans_norm'] = kep_response.transmission / np.sum(kep_response.transmission)
kep_response = kep_response.reset_index(drop=True)

In [ ]:
Kepler_filter_curve = Spectrum1D(spectral_axis=kep_response.wl_nm.values*u.nm, 
           flux=kep_response.transmission.values*u.dimensionless_unscaled)

## Create heat maps

In [ ]:
columns = [_ for _ in range(2300, 7100, 100)]
index = [_ for _ in range(7000, 2200, -100)]

### Kepler heat map

In [ ]:
Kepler_ratios = np.full((48,48), np.nan)

In [ ]:
for row in range(len(Kepler_ratios)): # Row
    for col in range(len(Kepler_ratios)): # Column
        if (col>=len(Kepler_ratios)-row-1):
            # get spectrum
            spec1 = PHOENIXSpectrum(teff=2300+(col*100), logg=4.5, metallicity=0, wl_lo=4000, wl_hi=12000, path=path)
            spec2 = PHOENIXSpectrum(teff=7000-(row*100), logg=4.5, metallicity=0, wl_lo=4000, wl_hi=12000, path=path)
            # resample
            resampled_spec1 = spec1.instrumental_broaden(resolving_power=300).resample(Kepler_filter_curve)
            resampled_spec2 = spec2.instrumental_broaden(resolving_power=300).resample(Kepler_filter_curve)
            # normalize
            normalization_scalar = np.percentile(resampled_spec2.flux, 95)
            normalized_spec1 = resampled_spec1.divide(normalization_scalar)
            normalized_spec2 = resampled_spec2.divide(normalization_scalar)
            # calculate ratio
            area_under_the_curve1 = np.sum(normalized_spec1.flux*Kepler_filter_curve.flux)
            area_under_the_curve2 = np.sum(normalized_spec2.flux*Kepler_filter_curve.flux)
            ratio = area_under_the_curve2/area_under_the_curve1
            # add to matrix
            Kepler_ratios[row][col] = ratio

In [ ]:
# convert array into dataframe
Kepler_ratios_df = pd.DataFrame(Kepler_ratios, columns=columns, index=index)

# save the dataframe as a csv file
Kepler_ratios_df.to_csv("../../data/Resampled Data/Kepler_ratios.csv", index=False, header=False)

In [ ]:
ax = plt.subplot(111)
ax.imshow(Kepler_ratios)
ax.set_xlabel('$T_{\mathrm{amb}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Kepler $T_{\mathrm{amb}}$ vs. $T_{\mathrm{spot}}$ (K)')

### TESS heat map

In [ ]:
TESS_ratios = np.full((48,48), np.nan)

In [ ]:
for row in range(len(TESS_ratios)): # Row
    for col in range(len(TESS_ratios)): # Column
        if (col>=len(TESS_ratios)-row-1):
            # get spectrum
            spec1 = PHOENIXSpectrum(teff=2300+(col*100), logg=4.5, metallicity=0, wl_lo=4000, wl_hi=12000, path=path)
            spec2 = PHOENIXSpectrum(teff=7000-(row*100), logg=4.5, metallicity=0, wl_lo=4000, wl_hi=12000, path=path)
            # resample
            resampled_spec1 = spec1.instrumental_broaden(resolving_power=300).resample(TESS_filter_curve)
            resampled_spec2 = spec2.instrumental_broaden(resolving_power=300).resample(TESS_filter_curve)
            # normalize
            normalization_scalar = np.percentile(resampled_spec2.flux, 95)
            normalized_spec1 = resampled_spec1.divide(normalization_scalar)
            normalized_spec2 = resampled_spec2.divide(normalization_scalar)
            # calculate ratio
            area_under_the_curve1 = np.sum(normalized_spec1.flux*TESS_filter_curve.flux)
            area_under_the_curve2 = np.sum(normalized_spec2.flux*TESS_filter_curve.flux)
            ratio = area_under_the_curve2/area_under_the_curve1
            # add to matrix
            TESS_ratios[row][col] = ratio

In [ ]:
# convert array into dataframe
TESS_ratios_df = pd.DataFrame(TESS_ratios, columns=columns, index=index)

# save the dataframe as a csv file
TESS_ratios_df.to_csv("../../data/Resampled Data/TESS_ratios.csv", index=False, header=False)

In [ ]:
ax = plt.subplot(111)
ax.imshow(TESS_ratios)
ax.set_xlabel('$T_{\mathrm{amb}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('TESS $T_{\mathrm{amb}}$ vs. $T_{\mathrm{spot}}$ (K)')

### Ratio of TESS over Kepler

In [ ]:
TESS_v_Kepler_ratios = np.full((48,48), np.nan)

In [ ]:
for row in range(len(TESS_v_Kepler_ratios)): # Row
    for col in range(len(TESS_v_Kepler_ratios)): # Column
        if (col>=len(TESS_v_Kepler_ratios)-row-1):
            TESS_v_Kepler_ratios[row][col] = TESS_ratios[row][col]/Kepler_ratios[row][col]

In [ ]:
# convert array into dataframe
DF = pd.DataFrame(TESS_v_Kepler_ratios)
  
# save the dataframe as a csv file
DF.to_csv("../../data/Resampled Data/TESS_v_Kepler_ratios.csv", index=False, header=False)

In [ ]:
ax = plt.subplot(111)
ax.imshow(TESS_v_Kepler_ratios)
ax.set_xlabel('$T_{\mathrm{amb}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Ratio of TESS over Kepler')

### Ratio of 1-TESS over 1-Kepler

In [ ]:
one_minus_ratio = np.full((48,48), np.nan)

In [ ]:
Kepler_ratios = pd.read_csv('../../data/Resampled Data/Kepler_ratios.csv', names=columns)
TESS_ratios = pd.read_csv('../../data/Resampled Data/TESS_ratios.csv', names=columns)

In [ ]:
if type(Kepler_ratios) == pd.core.frame.DataFrame:
    Kepler_ratios = Kepler_ratios.to_numpy()
if type(TESS_ratios) == pd.core.frame.DataFrame:
    TESS_ratios = TESS_ratios.to_numpy()

In [ ]:
for row in range(len(one_minus_ratio)): # Row
    for col in range(len(one_minus_ratio)): # Column
        if (col>=len(one_minus_ratio)-row-1):
            if 1-TESS_ratios[row][col] == 0 and 1-Kepler_ratios[row][col] == 0:
                one_minus_ratio[row][col] = np.NaN
            else:
                one_minus_ratio[row][col] = (1-TESS_ratios[row][col])/(1-Kepler_ratios[row][col])

In [ ]:
# convert array into dataframe
DF = pd.DataFrame(one_minus_ratio)
  
# save the dataframe as a csv file
DF.to_csv("../../data/Resampled Data/one_minus_ratio.csv", index=False, header=False)

In [ ]:
ax = plt.subplot(111)
ax.imshow(one_minus_ratio)
ax.set_xlabel('$T_{\mathrm{amb}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Ratio of 1-TESS over 1-Kepler')

## Generate graphs with CSV's

In [ ]:
Kepler_ratios = pd.read_csv('../../data/Resampled Data/Kepler_ratios.csv')
TESS_ratios = pd.read_csv('../../data/Resampled Data/TESS_ratios.csv')
TESS_v_Kepler_ratios = pd.read_csv('../../data/Resampled Data/TESS_v_Kepler_ratios.csv')
one_minus_ratio = pd.read_csv('../../data/Resampled Data/one_minus_ratio.csv')

### Kepler

In [ ]:
ax = plt.subplot()
im = ax.imshow(Kepler_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Kepler $T_{\mathrm{phot}}$ vs. $T_{\mathrm{spot}}$ (K)')

# cax.set_title('Contrast', rotation=270)

plt.savefig('../../papers/paper1/figures/Resampled Contrast/Kepler_contrast.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### TESS

In [ ]:
ax = plt.subplot()
im = ax.imshow(TESS_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('TESS $T_{\mathrm{phot}}$ vs. $T_{\mathrm{spot}}$ (K)')

# cax.set_title('Contrast', rotation=270)

plt.savefig('../../papers/paper1/figures/Resampled Contrast/TESS_contrast.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### Ratio of TESS over Kepler

In [ ]:
ax = plt.subplot()
im = ax.imshow(TESS_v_Kepler_ratios, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Ratio of TESS over Kepler')

plt.savefig('../../papers/paper1/figures/Resampled Contrast/TESS_v_Kepler_ratios.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

### Ratio of 1-TESS over 1-Kepler

In [ ]:
ax = plt.subplot()
im = ax.imshow(one_minus_ratio, origin='upper', extent=(2300, 7000, 2300, 7000), aspect=1, vmin=0, vmax=1)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='5%', pad=0.05)
plt.colorbar(im, cax=cax)

x = np.linspace(2300, 6300, 100)
y = -1.1668835503435e-4*x**2 + 1.374977439464*x - 567.8141350083
ax.plot(x, y, color='red', linestyle='dashed', label='Fang et. al. 2015')

x = np.linspace(2300, 3300, 100)
y = 0.86*x
ax.plot(x, y, color='white', linestyle='dashed', label='Rackham 2018')

x = np.linspace(3880, 6300, 100)
y = 0.418*x + 1620
ax.plot(x, y, color='black', linestyle='dashed', label='Rackham 2019')

ax.set_xlim(2300, 6300)
ax.set_ylim(2300, 6300)

ax.legend(loc='upper left')
ax.set_xlabel('$T_{\mathrm{phot}}$ (K)')
ax.set_ylabel('$T_{\mathrm{spot}}$ (K)')
ax.set_title('Ratio of 1-TESS over 1-Kepler')

plt.savefig('../../papers/paper1/figures/Resampled Contrast/one_minus_ratio.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)